# Download and Transcribe Playlist

## Set up Environment

In [ ]:
#@title Specify Requirements
%%writefile requirements.txt
accelerate
camel_converter
optimum
polars >=0.19
pandas>=2.1.3
pytube
transformers

Writing requirements.txt


In [ ]:
#@title Install Requirements
!pip install -Ur requirements.txt &>/content/logs.txt

In [ ]:
#@title Import Modules
from pytube import Playlist, YouTube
from camel_converter import to_snake
from tqdm.notebook import tqdm
from rich.pretty import pprint
import pandas as pd
import os

In [ ]:
#@title Load Pipeline
import torch
from transformers import pipeline
from transformers.utils import is_flash_attn_2_available

# Select checkpoint from https://huggingface.co/openai/whisper-large-v3#model-details
model_id = "distil-whisper/large-v2"
pipe = pipeline(
    "automatic-speech-recognition",
    model=model_id,
    torch_dtype=torch.float16,
    device="cuda:0", # or mps for Mac devices
    model_kwargs={"use_flash_attention_2": is_flash_attn_2_available()},
)
pprint(pipe.model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WhisperForConditionalGeneration(
  (model): WhisperModel(
│   (encoder): WhisperEncoder(
│     (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
│     (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
│     (embed_positions): Embedding(1500, 1280)
│     (layers): ModuleList(
│   │   (0-31): 32 x WhisperEncoderLayer(
│   │     (self_attn): WhisperAttention(
│   │   │   (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
│   │   │   (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │   │   (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │   │   (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │     )
│   │     (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   │     (activation_fn): GELUActivation()
│   │     (fc1): Linear(in_features=1280, out_features=5120, bias=True)
│   │     (fc2): Linear(in_features=5120, out_features=1280, bias=True)
│   │     (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   │   )
│     )
│     (layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   )
│   (decoder): WhisperDecoder(
│     (embed_tokens): Embedding(51865, 1280, padding_idx=50257)
│     (embed_positions): WhisperPositionalEmbedding(448, 1280)
│     (layers): ModuleList(
│   │   (0-1): 2 x WhisperDecoderLayer(
│   │     (self_attn): WhisperAttention(
│   │   │   (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
│   │   │   (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │   │   (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │   │   (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │     )
│   │     (activation_fn): GELUActivation()
│   │     (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   │     (encoder_attn): WhisperAttention(
│   │   │   (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
│   │   │   (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │   │   (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │   │   (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
│   │     )
│   │     (encoder_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   │     (fc1): Linear(in_features=1280, out_features=5120, bias=True)
│   │     (fc2): Linear(in_features=5120, out_features=1280, bias=True)
│   │     (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   │   )
│     )
│     (layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
│   )
  )
  (proj_out): Linear(in_features=1280, out_features=51865, bias=False)
)

In [ ]:
#@title Define Utility Functions
def download_audio(video_url):
  """Download youtube video and return videoDetails"""
  yt = YouTube(video_url)
  (yt.streams
    .filter(only_audio=True, file_extension = "mp4")
    .order_by("abr")
    .desc()
    .first()
    .download(filename = f"videos/{yt.video_id}.mp4"))
  return yt.vid_info['videoDetails']

def transcribe_audio(video_id):
  """Transcribe downloaded video"""
  outputs = pipe(
      f"videos/{video_id}.mp4",
      chunk_length_s=30,
      batch_size=24,
      return_timestamps=True,
  )
  return outputs

## Download and Transcribe

In [ ]:
#@title List Videos in Playlist
import os
os.makedirs("videos", exist_ok=True)
playlist_url = "https://www.youtube.com/playlist?list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo"
video_urls = Playlist(playlist_url)
pprint(list(video_urls), max_length=3)

[
│   'https://www.youtube.com/watch?v=GY_ntGiFZ38',
│   'https://www.youtube.com/watch?v=OSV7cxma6_s',
│   'https://www.youtube.com/watch?v=IfZR0TiO9zQ',
│   ... +400
]

In [ ]:
#@title Download Audio
video_info = download_audio(video_urls[0])
pprint(video_info)

{
│   'videoId': 'GY_ntGiFZ38',
│   'title': 'Only Way To Unf*ck Your Life - How To Get 1% Better Everyday & Accomplish Anything In 2024',
│   'lengthSeconds': '6915',
│   'keywords': [
│   │   'Tom Bilyeu',
│   │   'Impact Theory',
│   │   'ImpactTheory',
│   │   'TomBilyeu',
│   │   'Inside Quest',
│   │   'InsideQuest',
│   │   'Tom Bilyou',
│   │   'Theory Impact',
│   │   'motivation',
│   │   'inspiration',
│   │   'talk show',
│   │   'interview',
│   │   'motivational speech'
│   ],
│   'channelId': 'UCnYMOamNKLGVlJgRUbamveA',
│   'isOwnerViewing': False,
│   'shortDescription': "For a limited time, Facet will waive the $250 enrollment fee for new annual members, visit https://bit.ly/impactfacet to learn more!\n\nVanta is offering Impact Theory listeners a free trial when you sign up at https://bit.ly/ImpactVanta.\n\nI am honored to sit down with the brilliant mind of Adam Grant, author of Hidden Potential, to uncover the secrets to unleashing your true greatness.\n\nIn a world where natural talent is often overvalued, Adam Grant reminds us that relying solely on our innate abilities is not enough. He challenges us to discard the notion of mindlessly putting in 10,000 hours and instead encourages us to work smarter, not harder. But don't be mistaken, this conversation is not just about working smarter; it's about becoming legendary.\n\nJoin us as we explore the revolutionary idea of throwing away what we think we know and embarking on a real path to mastery. It's time to unf*ck your life and unlock your hidden potential.\n\nThis epic conversation with Adam will guide you on how to get 1% better every day and avoid the tragic fate of leaving your untapped potential trapped inside. Together, we will beat the odds and shatter the limitations that hold us back.\n\nBut that's not all and if you're ready for lasting change and eager to act as if you can transform anything you set your mind to, then this episode is a must-watch. Get ready to challenge the great myth of potential and discover what truly leaves people dormant.\n\nSo, join me, Tom Bilyeu, and the extraordinary Adam Grant as we redefine success, debunk misconceptions, and illuminate the path to unlocking your hidden potential. This is an episode you won't want to miss!\n\nIf you enjoyed today’s conversation, be sure to check out Adam’s book, Hidden Potential: https://www.amazon.com/Hidden-Potential-Science-Achieving-Greater/dp/0593653149 \nXKCD Free Speech Comic: https://xkcd.com/1357/ \n\nChapter Markers:\n[00:00] Introduction\n[00:31] The Controversial Mindset\n[11:03] Hard Work Isn’t Enough\n[23:58] Hustle Culture \n[31:57] Extraordinary Like Steph Curry\n[46:14] The Future of Gamified Work\n[59:08] Downsizing Potential with Elon\n[1:21:26] Embrace Tension, Ignite Potential\n[1:39:32] Maxing Potential With A.I.\n\n\nWisdom with Adam Grant:\n\n“A growth mindset is not a magic bullet, it’s not a panacea, but the absence of one can be devastating.”\n\n“It’s not about how hard you work, it’s about how well you learn.”\n\n“Great minds don’t think alike, they challenge each other to think differently.”\n\n\nFollow Adam Grant:\nWebsite: https://adamgrant.net/ \nTwitter: https://twitter.com/adammgrant \nInstagram: https://www.instagram.com/adamgrant/ \n\nFollow Me Tom Bilyeu: \nWebsite: https://impacttheoryuniversity.com/ \nTwitter: https://twitter.com/TomBilyeu \nInstagram: https://www.instagram.com/tombilyeu/",
│   'isCrawlable': True,
│   'thumbnail': {
│   │   'thumbnails': [
│   │   │   {'url': 'https://i.ytimg.com/vi/GY_ntGiFZ38/default.jpg', 'width': 120, 'height': 90},
│   │   │   {'url': 'https://i.ytimg.com/vi/GY_ntGiFZ38/mqdefault.jpg', 'width': 320, 'height': 180},
│   │   │   {'url': 'https://i.ytimg.com/vi/GY_ntGiFZ38/hqdefault.jpg', 'width': 480, 'height': 360},
│   │   │   {'url': 'https://i.ytimg.com/vi/GY_ntGiFZ38/sddefault.jpg', 'width': 640, 'height': 480}
│   │   ]
│   },
│   'allowRatings': True,
│   'viewCount': '43665',
│   'author': 'Tom Bilye

In [ ]:
#@title Transcribe Audio
transcript = transcribe_audio(video_info['videoId'])
pprint(transcript['chunks'], max_length=2)

[
│   {
│   │   'timestamp': (0.0, 6.0),
│   │   'text': " You're going to fail if your plan is to rely on natural talent or mindlessly clocking 10,000 hours."
│   },
│   {
│   │   'timestamp': (6.0, 11.0),
│   │   'text': " If you want an outrageous amount of success, you're going to have to throw away most of what you know"
│   },
│   ... +1604
]

In [ ]:
#@title Save Info
from pathlib import Path
import json
import os
os.makedirs('data', exist_ok=True)
episode_info = video_info | {"transcript": transcript["chunks"]}
with Path(f"data/{video_info['videoId']}.json").open("w", encoding="UTF-8") as target:
    json.dump(episode_info, target)